<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9afec3ad245d81ec94220cf320ab7094c719d3bb376c8f459880157dd2a355bc
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-08 13:45:30
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: 54.98 K (0.39%)
Current PnL: -17.83 L (-11.99%)
CY Booked + Current PnL: -7.97 L (-5.36%)
-------------------
Total profit:  2.88 L
Total loss:  -20.71 L
-------------------
Total Booked + Current PnL: 19.01 L (15.62%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.03%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.54 L (56.01%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.9%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-27.95,72.0,H-MC,8.01,144152.0,29216.0,7366.0,0.92,25.42,5.11,31.83,81.0,3.97,1.08,45.35,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-14.08,58.0,H-LC,10.48,171567.0,15390.0,10706.0,-0.08,9.85,6.24,16.71,22.0,1.44,1.29,29.01,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,104.70,62.0,M-SC,6.19,88163.0,-12614.0,12704.0,-0.98,-12.52,14.41,0.09,245.0,-0.99,0.66,15.83,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,50.0,M-LC,4.53,214809.0,13330.0,13877.0,0.50,6.62,6.46,13.51,60.0,0.96,1.62,20.05,XY25,NTT,BANKS
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.05,192202.0,41289.0,16010.0,2.40,27.36,8.33,37.97,93.0,2.58,1.45,59.59,AR,ATH,AUTO


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HAVELLS,2069.16,-0.09,62.0,H-MC,1.89,247806.0,75.0,74986.0,0.38,0.03,30.26,30.30,92.0,0.00,1.86,13.33,X40,ATH,ELECTRICAL
36,ICICIGI,2260.25,-20.81,41.0,H-MC,3.95,136315.0,182.0,30944.0,0.09,0.13,22.70,22.86,91.0,0.01,1.03,12.76,X40,ATH,INSURANCE
53,PGHH,17973.08,-30.81,52.0,H-MC,4.88,200115.0,-705.0,69480.0,-1.13,-0.35,34.72,34.25,80.0,-0.01,1.51,4.83,X40,ATH,FMCG
88,WIPRO,420.00,-14.25,39.0,M-LC,6.89,150598.0,-347.0,109801.0,-0.32,-0.23,72.91,72.51,53.0,-0.00,1.13,5.78,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-2.80,31.0,H-LC,10.06,221543.0,-20381.0,83655.0,-0.56,-8.42,37.76,26.15,8.0,-0.24,1.67,4.69,X40,ATH,IT
41,INFY,2275.00,-21.75,37.0,H-LC,10.88,305527.0,-7361.0,179039.0,-0.71,-2.35,58.60,54.87,3.0,-0.04,2.30,4.14,X40,BTT,IT
76,TCS,4998.00,-28.69,38.0,H-LC,14.29,284341.0,-61615.0,185476.0,-0.77,-17.81,65.23,35.80,1.0,-0.33,2.14,0.73,X40,BTT,IT
85,VBL,671.64,-15.98,39.0,H-LC,6.90,302827.0,-13015.0,125673.0,1.06,-4.12,41.50,35.67,5.0,-0.10,2.28,9.16,X40N,ATH,FMCG
82,UNITDSPR,1644.00,-14.66,43.0,H-LC,8.43,171719.0,-10040.0,45282.0,-0.85,-5.52,26.37,19.39,86.0,-0.22,1.29,2.17,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-6.65,48.0,H-LC,1.73,231772.0,-14379.0,93590.0,-0.87,-5.84,40.38,32.18,16.0,-0.15,1.74,28.10,X200,ATH,IT
52,NESTLEIND,1377.00,-9.50,56.0,H-LC,2.72,278600.0,13174.0,43629.0,-1.48,4.96,15.66,21.40,11.0,0.30,2.10,12.41,XY25,NTT,FMCG
43,ITC,452.00,-38.34,48.0,H-LC,3.13,199439.0,-699.0,21599.0,0.12,-0.35,10.83,10.44,4.0,-0.03,1.50,5.19,X40,NTT,FMCG
19,CIPLA,1795.00,-20.12,49.0,H-LC,5.76,212761.0,8261.0,33148.0,-0.03,4.04,15.58,20.25,10.0,0.25,1.60,13.13,X40N,BTT,PHARMA
85,VBL,671.64,-15.98,39.0,H-LC,6.90,302827.0,-13015.0,125673.0,1.06,-4.12,41.50,35.67,5.0,-0.10,2.28,9.16,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.0,-35.61,66.0,L-SC,38.90,96000.0,-17549.0,57600.0,12.21,-15.45,60.00,35.27,268.0,-0.30,0.72,134.96,XR,NTT,HOTELS
66,SHALBY,327.0,1104.15,79.0,M-SC,3.23,163804.0,-17339.0,62802.0,7.71,-9.57,38.34,25.10,235.0,-0.28,1.23,31.27,XY24,NTT,HEALTHCARE
55,RAJESHEXPO,518.0,1763.64,59.0,L-SC,8.26,50846.0,-86331.0,86423.0,6.12,-62.93,169.97,0.07,267.0,-1.00,0.38,26.45,OX40N,NTT,JEWELLERY
84,VALIANTORG,838.0,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS
75,TATAMOTORS,1065.0,-17.55,65.0,M-LC,2.57,260217.0,-16306.0,126387.0,3.64,-5.90,48.57,39.81,54.0,-0.13,1.96,24.71,XY24,NTT,AUTO


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,9.48,53.0,H-MC,3.32,189050.0,24380.0,43595.0,-2.73,14.81,23.06,41.28,89.0,0.56,1.42,23.97,X40N,ATH,AC
48,LAOPALA,464.00,109.37,44.0,H-SC,3.87,66557.0,-34289.0,60114.0,-2.05,-34.00,90.32,25.61,142.0,-0.57,0.50,23.40,AR,NTT,CERAMICS
5,ASIANPAINT,4250.00,-11.36,56.0,H-LC,8.30,223590.0,-28178.0,150409.0,-1.52,-11.19,67.27,48.55,27.0,-0.19,1.68,20.27,X40,BTT,PAINTS
52,NESTLEIND,1377.00,-9.50,56.0,H-LC,2.72,278600.0,13174.0,43629.0,-1.48,4.96,15.66,21.40,11.0,0.30,2.10,12.41,XY25,NTT,FMCG
7,ATULAUTO,844.00,3921.62,75.0,M-SC,8.65,125328.0,-22040.0,73856.0,-1.43,-14.96,58.93,35.16,236.0,-0.30,0.94,28.01,XY24,NTT,AUTO


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,104.70,62.0,M-SC,6.19,88163.0,-12614.0,12704.0,-0.98,-12.52,14.41,0.09,245.0,-0.99,0.66,15.83,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,87.86,52.0,H-SC,12.93,118692.0,-23787.0,43311.0,3.27,-16.70,36.49,13.70,163.0,-0.55,0.89,40.09,OX40N,NTT,BANKS
46,KANSAINER,340.0,-67.97,57.0,H-SC,3.24,220392.0,-49275.0,85600.0,0.82,-18.27,38.84,13.47,138.0,-0.58,1.66,11.47,XY24,NTT,PAINTS
39,INDIGOPNTS,1408.0,114.84,50.0,M-SC,3.54,141062.0,-33497.0,33530.0,-0.68,-19.19,23.77,0.02,221.0,-1.00,1.06,22.70,OX40N,NTT,PAINTS
18,CERA,9475.0,-20.29,44.0,H-SC,2.55,113814.0,-29989.0,56736.0,-0.28,-20.85,49.85,18.60,149.0,-0.53,0.86,25.60,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-28.69,38.0,H-LC,14.29,284341.0,-61615.0,185476.0,-0.77,-17.81,65.23,35.80,1.0,-0.33,2.14,0.73,X40,BTT,IT
41,INFY,2275.00,-21.75,37.0,H-LC,10.88,305527.0,-7361.0,179039.0,-0.71,-2.35,58.60,54.87,3.0,-0.04,2.30,4.14,X40,BTT,IT
43,ITC,452.00,-38.34,48.0,H-LC,3.13,199439.0,-699.0,21599.0,0.12,-0.35,10.83,10.44,4.0,-0.03,1.50,5.19,X40,NTT,FMCG
85,VBL,671.64,-15.98,39.0,H-LC,6.90,302827.0,-13015.0,125673.0,1.06,-4.12,41.50,35.67,5.0,-0.10,2.28,9.16,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,47.0,H-LC,12.91,247133.0,-14486.0,133699.0,0.61,-5.54,54.10,45.57,7.0,-0.11,1.86,5.16,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.05,192202.0,41289.0,16010.0,2.40,27.36,8.33,37.97,93.0,2.58,1.45,59.59,AR,ATH,AUTO
25,DIXON,18940.15,-27.95,72.0,H-MC,8.01,144152.0,29216.0,7366.0,0.92,25.42,5.11,31.83,81.0,3.97,1.08,45.35,X40N,ATH,IT
26,DMART,5201.00,-7.27,62.0,H-LC,16.05,231589.0,47108.0,23252.0,-1.40,25.54,10.04,38.14,38.0,2.03,1.74,38.85,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,45.15,65.0,M-MC,7.54,228716.0,3754.0,163898.0,2.75,1.67,71.66,74.53,192.0,0.02,1.72,32.79,XY24,BTT,STEEL
14,BLUESTARCO,2326.38,9.48,53.0,H-MC,3.32,189050.0,24380.0,43595.0,-2.73,14.81,23.06,41.28,89.0,0.56,1.42,23.97,X40N,ATH,AC
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.05,192202.0,41289.0,16010.0,2.40,27.36,8.33,37.97,93.0,2.58,1.45,59.59,AR,ATH,AUTO
86,VOLTAS,1918.49,1.44,61.0,H-MC,3.69,212910.0,21168.0,74859.0,-0.94,11.04,35.16,50.08,99.0,0.28,1.60,18.59,XY25,ATH,AC
31,HAVELLS,2069.16,-0.09,62.0,H-MC,1.89,247806.0,75.0,74986.0,0.38,0.03,30.26,30.30,92.0,0.00,1.86,13.33,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.05,192202.0,41289.0,16010.0,2.40,27.36,8.33,37.97,93.0,2.58,1.45,59.59,AR,ATH,AUTO
51,NATIONALUM,247.44,-42.83,76.0,H-MC,1.29,113478.0,12974.0,17669.0,0.93,12.91,15.57,30.49,79.0,0.73,0.85,49.50,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.40,62.0,M-SC,3.66,102458.0,10960.0,67796.0,0.67,11.98,66.17,86.07,223.0,0.16,0.77,48.79,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.09,49.0,H-SC,12.10,136422.0,13086.0,120679.0,0.74,10.61,88.46,108.45,119.0,0.11,1.03,35.33,AR,ATH,MISC
88,WIPRO,420.00,-14.25,39.0,M-LC,6.89,150598.0,-347.0,109801.0,-0.32,-0.23,72.91,72.51,53.0,-0.00,1.13,5.78,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-5.18,43.0,H-SC,15.37,88253.0,-12710.0,107236.0,-1.02,-12.59,121.51,93.63,148.0,-0.12,0.66,31.45,SR,ATH,CHEMICALS
50,MASFIN,398.61,-18.25,51.0,H-SC,12.87,93075.0,-4905.0,26508.0,-0.16,-5.01,28.48,22.05,152.0,-0.19,0.70,35.29,XR,ATH,FINANCE
21,COFFEEDAY,80.00,-35.61,66.0,L-SC,38.90,96000.0,-17549.0,57600.0,12.21,-15.45,60.00,35.27,268.0,-0.30,0.72,134.96,XR,NTT,HOTELS
84,VALIANTORG,838.00,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS
51,NATIONALUM,247.44,-42.83,76.0,H-MC,1.29,113478.0,12974.0,17669.0,0.93,12.91,15.57,30.49,79.0,0.73,0.85,49.50,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-35.61,66.0,L-SC,38.90,96000.0,-17549.0,57600.0,12.21,-15.45,60.00,35.27,268.0,-0.30,0.72,134.96,XR,NTT,HOTELS
7,ATULAUTO,844.00,3921.62,75.0,M-SC,8.65,125328.0,-22040.0,73856.0,-1.43,-14.96,58.93,35.16,236.0,-0.30,0.94,28.01,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.05,192202.0,41289.0,16010.0,2.40,27.36,8.33,37.97,93.0,2.58,1.45,59.59,AR,ATH,AUTO
62,SAMMAANCAP,326.00,-171.59,66.0,M-SC,5.40,82938.0,-19272.0,112663.0,0.32,-18.86,135.84,91.37,208.0,-0.17,0.62,34.64,XY25,NTT,FINANCE
66,SHALBY,327.00,1104.15,79.0,M-SC,3.23,163804.0,-17339.0,62802.0,7.71,-9.57,38.34,25.10,235.0,-0.28,1.23,31.27,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.70
1,25,43.55
2,50,73.80


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.24
LC    35.97
MC    25.79
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.09
X40      15.47
X40N     14.18
XY25     10.05
AR        9.54
XR        9.16
OX40N     7.47
MH        1.77
X200      1.74
X5K       1.44
SR        1.09
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.63
H-MC    22.74
H-SC    22.67
M-SC    13.91
M-LC     7.25
M-MC     2.73
L-SC     1.66
L-LC     1.09
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.75,-0.88,35.50
IT,13.31,-16.72,74.59
FINANCE,7.42,-22.15,67.19
BANKS,7.42,-15.59,64.67
PAINTS,6.04,-15.04,40.79
MISC,5.88,-17.83,85.38
ELECTRICAL,5.70,-8.64,68.52
AUTO,4.95,-2.80,47.52
AC,3.75,3.06,29.76


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2729516.0,22
AR,1213211.0,10
XR,1074020.0,13
X40,911913.0,10
X40N,547025.0,11
OX40N,464483.0,10
XY25,415883.0,7
SR,197828.0,2
X5K,126734.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2762418.0,27
H-LC,1447023.0,16
M-SC,1394688.0,17
H-MC,1179527.0,17
M-LC,416155.0,5
M-MC,314280.0,2
L-SC,240245.0,3
L-MC,59712.0,1
L-LC,39852.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,926497.0,6
M-SC,XY24,811545.0,7
H-SC,AR,663162.0,5
H-LC,X40,620178.0,5
H-SC,XR,516626.0,6
H-MC,XY24,459089.0,4
H-LC,AR,375639.0,2
M-MC,XY24,314280.0,2
H-LC,X40N,238061.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
